# Get Dairy Queen locations

#### Load Python tools and Jupyter config

In [1]:
%load_ext lab_black

In [31]:
import json
import requests
import pandas as pd
import geopandas as gpd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm, trange
import urllib.request
import usaddress

In [3]:
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000
pd.options.display.max_colwidth = None

## Read data

#### All the locations

In [4]:
states = [
    "AK",
    "AL",
    "AR",
    "AZ",
    "CA",
    "CO",
    "CT",
    "DE",
    "FL",
    "GA",
    "HI",
    "IA",
    "ID",
    "IL",
    "IN",
    "KS",
    "KY",
    "LA",
    "MA",
    "MD",
    "ME",
    "MI",
    "MN",
    "MO",
    "MS",
    "MT",
    "NC",
    "ND",
    "NE",
    "NH",
    "NJ",
    "NM",
    "NV",
    "NY",
    "OH",
    "OK",
    "OR",
    "PA",
    "RI",
    "SC",
    "SD",
    "TN",
    "TX",
    "UT",
    "VA",
    "WA",
    "WI",
    "WV",
    "WY",
]

In [5]:
state_jsons = []

for s in states:
    state_jsons.append(
        f"https://www.dairyqueen.com/_next/data/-6M6OZM5wEcP6PTvorQ70/en-us/locations/{s.lower()}.json"
    )

In [6]:
pages = []

for j in state_jsons:
    with urllib.request.urlopen(f"{j}") as url:
        data = json.load(url)
        location_dict = {
            "state": data["pageProps"]["navContext"]["locationInfo"]["cities"]["state"],
            "cities": data["pageProps"]["navContext"]["locationInfo"]["cities"][
                "cities"
            ],
        }
        df = pd.DataFrame(location_dict)
        pages.append(df)

In [7]:
city_state = pd.concat(pages).reset_index(drop=True)

In [18]:
city_state

,state,cities,url
0,ak,Anchorage,https://www.dairyqueen.com/_next/data/-6M6OZM5wEcP6PTvorQ70/en-us/locations/ak/anchorage.json
1,ak,Palmer,https://www.dairyqueen.com/_next/data/-6M6OZM5wEcP6PTvorQ70/en-us/locations/ak/palmer.json
2,ak,Soldotna,https://www.dairyqueen.com/_next/data/-6M6OZM5wEcP6PTvorQ70/en-us/locations/ak/soldotna.json
3,ak,Wasilla,https://www.dairyqueen.com/_next/data/-6M6OZM5wEcP6PTvorQ70/en-us/locations/ak/wasilla.json
4,al,Alabaster,https://www.dairyqueen.com/_next/data/-6M6OZM5wEcP6PTvorQ70/en-us/locations/al/alabaster.json
...,...,...,...
3348,wy,Jackson,https://www.dairyqueen.com/_next/data/-6M6OZM5wEcP6PTvorQ70/en-us/locations/wy/jackson.json
3349,wy,Laramie,https://www.dairyqueen.com/_next/data/-6M6OZM5wEcP6PTvorQ70/en-us/locations/wy/laramie.json
3350,wy,Powell,https://www.dairyqueen.com/_next/data/-6M6OZM5wEcP6PTvorQ70/en-us/locations/wy/powell.json
3351,wy,Riverton,https://www.dairyqueen.com/_next/data/-6M6OZM5wEcP6PTvorQ70/en-us/locations/wy/riverton.json


In [27]:
city_state["url"] = (
    f"https://www.dairyqueen.com/en-us/locations"
    + "/"
    + city_state["state"]
    + "/"
    + city_state["cities"].str.lower()
)

In [28]:
city_urls = list(city_state["url"].unique())

In [29]:
city_urls[0:3]

['https://www.dairyqueen.com/en-us/locations/ak/anchorage',
 'https://www.dairyqueen.com/en-us/locations/ak/palmer',
 'https://www.dairyqueen.com/en-us/locations/ak/soldotna']

In [36]:
response = requests.get("https://www.dairyqueen.com/en-us/locations/ak/anchorage/")

In [37]:
soup = BeautifulSoup(response.text, "html.parser")

In [44]:
soup.findAll("a")

[<a aria-label="Skip to main content" class="SkipLinks_skip__link__Z1Uil" href="#skip-to-content"><span class="" data-lang-var-key="SKIP_TO_CONTENT">Skip to Content</span></a>,
 <a aria-label="Skip to sitemap" class="SkipLinks_skip__link__Z1Uil" href="/en-us/sitemap/" id="skip-to-sitemap"><span class="" data-lang-var-key="SKIP_TO_SITEMAP">Skip to Sitemap</span></a>,
 <a class="Breadcrumbs_breadcrumb__link__OMirL hover:underline" data-cy="breadcrumbs-link-home" href="/en-us/">Home</a>,
 <a class="Breadcrumbs_breadcrumb__link__OMirL hover:underline" data-cy="breadcrumbs-link-locations" href="/en-us/locations/">Locations</a>,
 <a class="Breadcrumbs_breadcrumb__link__OMirL hover:underline" data-cy="breadcrumbs-link-alaska" href="/en-us/locations/ak/">Alaska</a>,
 <a class="logo p-0 flex-shrink-0" data-cy="logo-link" href="/en-us/"><img aria-label="Dairy Queen Home" class="Header_brand-logo__AHo08" height="55px" src="https://dairyqueen-prod.dotcdn.io/application/themes/dairyqueen/assets/ima